# World Bank Ethiopia Capstone Project

## Stage 1: Data Collection and Cleaning

### Project Overview
This notebook performs the initial data acquisition and cleaning phase for the World Bank Ethiopia macroeconomic analysis project. We will load raw World Development Indicators (WDI) data for Ethiopia and transform it into a clean, analysis-ready dataset.

### Objectives
1. **Data Acquisition**: Load raw World Bank WDI data for Ethiopia
2. **Data Inspection**: Understand the structure, data types, and quality of the raw data
3. **Data Cleaning**: Handle missing values, standardize column names, and ensure data consistency
4. **Data Validation**: Verify data quality and completeness
5. **Data Export**: Save cleaned dataset for downstream analysis

### Key Indicators
The dataset contains the following macroeconomic indicators:
- **GDP (constant 2015 US$)**: Total economic output adjusted for inflation
- **GDP growth (annual %)**: Year-over-year GDP growth rate
- **GDP per capita (constant 2015 US$)**: GDP divided by population
- **Inflation, consumer prices (annual %)**: Consumer price index annual change
- **Unemployment, total (% of total labor force)**: Unemployment rate

### Data Source
- **Source**: World Bank Open Data
- **Dataset**: World Development Indicators (WDI)
- **Country**: Ethiopia (ETH)
- **Time Period**: 1960-2024
- **File**: `datasets/raw/ethiopia_wdi_raw.csv`

## Step 1: Import Required Libraries

We begin by importing the necessary Python libraries for data manipulation and analysis.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("=" * 80)
print("STAGE 1: DATA COLLECTION AND CLEANING")
print("=" * 80)
print("\nLibraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## Step 2: Load Raw Dataset

Load the raw World Bank WDI data for Ethiopia. The raw data is in a wide format with years as columns.

In [ ]:
# Define file path
raw_data_path = '../datasets/raw/ethiopia_wdi_raw.csv'

# Check if file exists
if not os.path.exists(raw_data_path):
    raise FileNotFoundError(f"Raw data file not found at: {raw_data_path}")

# Load raw dataset
print(f"Loading raw data from: {raw_data_path}")
df_raw = pd.read_csv(raw_data_path)

print(f"\n✓ Data loaded successfully!")
print(f"  - Shape: {df_raw.shape[0]} rows × {df_raw.shape[1]} columns")
print(f"  - Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display first few rows
print("\n" + "=" * 80)
print("RAW DATA PREVIEW (First 5 rows):")
print("=" * 80)
display(df_raw.head())

## Step 3: Inspect Data Structure

Examine the structure of the raw data to understand:
- Column names and their meanings
- Data types
- Missing values
- Data quality issues

In [ ]:
# Display basic information about the dataset
print("=" * 80)
print("DATA STRUCTURE INFORMATION")
print("=" * 80)

print("\n1. Column Names:")
print("-" * 80)
for i, col in enumerate(df_raw.columns, 1):
    print(f"{i:2d}. {col}")

print("\n2. Data Types:")
print("-" * 80)
print(df_raw.dtypes)

print("\n3. Dataset Shape:")
print("-" * 80)
print(f"Rows: {df_raw.shape[0]}")
print(f"Columns: {df_raw.shape[1]}")

print("\n4. Missing Values Summary:")
print("-" * 80)
missing_summary = df_raw.isnull().sum()
missing_pct = (missing_summary / len(df_raw)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_summary,
    'Missing Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
if len(missing_df) > 0:
    display(missing_df)
else:
    print("No missing values found in the dataset.")

print("\n5. Sample Data (First 3 rows):")
print("-" * 80)
display(df_raw.head(3))

## Step 4: Transform Data from Wide to Long Format

The raw data is in wide format (years as columns). We need to transform it to long format (Year as a column) for time series analysis.

In [ ]:
# Identify key columns
id_cols = ['Country Name', 'Country Code', 'Series Name', 'Series Code']
year_cols = [col for col in df_raw.columns if '[YR' in col]

print(f"Found {len(year_cols)} year columns (1960-2024)")
print(f"Found {len(df_raw)} indicator series")

# Melt the dataframe from wide to long format
print("\nTransforming data from wide to long format...")
df_long = pd.melt(
    df_raw,
    id_vars=id_cols,
    value_vars=year_cols,
    var_name='Year_Str',
    value_name='Value'
)

# Extract year from column names (e.g., "1960 [YR1960]" -> 1960)
df_long['Year'] = df_long['Year_Str'].str.extract(r'(\d{4})').astype(int)
df_long = df_long.drop(columns=['Year_Str'])

print(f"✓ Transformation complete!")
print(f"  - Long format shape: {df_long.shape[0]} rows × {df_long.shape[1]} columns")

# Display sample of transformed data
print("\n" + "=" * 80)
print("TRANSFORMED DATA (Long Format) - Sample:")
print("=" * 80)
display(df_long.head(10))

## Step 5: Pivot to Create Indicator Columns

Transform the long format data into a wide format with each indicator as a separate column, suitable for time series analysis.

In [ ]:
# Create a mapping of series codes to clean column names
series_mapping = {
    'NY.GDP.MKTP.KD': 'gdp_constant',
    'NY.GDP.MKTP.KD.ZG': 'gdp_growth',
    'NY.GDP.PCAP.KD': 'gdp_per_capita',
    'FP.CPI.TOTL.ZG': 'inflation',
    'SL.UEM.TOTL.ZS': 'unemployment'
}

# Filter to only include indicators we need
df_filtered = df_long[df_long['Series Code'].isin(series_mapping.keys())].copy()

# Map series codes to clean names
df_filtered['Indicator'] = df_filtered['Series Code'].map(series_mapping)

# Pivot to create one column per indicator
print("Creating indicator columns...")
df_pivoted = df_filtered.pivot_table(
    index='Year',
    columns='Indicator',
    values='Value',
    aggfunc='first'  # Use first value if duplicates exist
)

# Reset index to make Year a column
df_pivoted = df_pivoted.reset_index()

# Sort by year
df_pivoted = df_pivoted.sort_values('Year').reset_index(drop=True)

print(f"✓ Pivot complete!")
print(f"  - Final shape: {df_pivoted.shape[0]} rows × {df_pivoted.shape[1]} columns")
print(f"  - Year range: {df_pivoted['Year'].min()} to {df_pivoted['Year'].max()}")

# Display sample
print("\n" + "=" * 80)
print("PIVOTED DATA (One column per indicator) - Sample:")
print("=" * 80)
display(df_pivoted.head(10))
display(df_pivoted.tail(10))

## Step 6: Data Cleaning and Validation

Perform comprehensive data cleaning:
- Handle missing values appropriately
- Convert data types
- Validate data ranges
- Check for outliers

In [ ]:
# Create a copy for cleaning
df_clean = df_pivoted.copy()

print("=" * 80)
print("DATA CLEANING AND VALIDATION")
print("=" * 80)

# 1. Check for missing values by indicator
print("\n1. Missing Values by Indicator:")
print("-" * 80)
missing_by_indicator = df_clean.isnull().sum()
missing_pct_by_indicator = (missing_by_indicator / len(df_clean)) * 100
missing_info = pd.DataFrame({
    'Missing Count': missing_by_indicator,
    'Missing Percentage': missing_pct_by_indicator
})
display(missing_info)

# 2. Convert numeric columns (handle '..' as missing values)
print("\n2. Converting data types and handling special values...")
numeric_cols = ['gdp_constant', 'gdp_growth', 'gdp_per_capita', 'inflation', 'unemployment']

for col in numeric_cols:
    if col in df_clean.columns:
        # Replace '..' and other non-numeric strings with NaN
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        print(f"  ✓ Converted {col} to numeric")

# 3. Data validation - check for reasonable ranges
print("\n3. Data Range Validation:")
print("-" * 80)
validation_results = {}
for col in numeric_cols:
    if col in df_clean.columns:
        non_null = df_clean[col].dropna()
        if len(non_null) > 0:
            validation_results[col] = {
                'Min': non_null.min(),
                'Max': non_null.max(),
                'Mean': non_null.mean(),
                'Non-null Count': len(non_null),
                'Null Count': df_clean[col].isnull().sum()
            }

validation_df = pd.DataFrame(validation_results).T
display(validation_df)

# 4. Check for obvious outliers (values that are clearly errors)
print("\n4. Outlier Detection (values outside 3 standard deviations):")
print("-" * 80)
for col in numeric_cols:
    if col in df_clean.columns:
        series = df_clean[col].dropna()
        if len(series) > 0:
            mean = series.mean()
            std = series.std()
            outliers = series[(series < mean - 3*std) | (series > mean + 3*std)]
            if len(outliers) > 0:
                print(f"\n  {col}: {len(outliers)} potential outliers")
                print(f"    Range: [{mean - 3*std:.2f}, {mean + 3*std:.2f}]")
                print(f"    Outlier values: {outliers.values[:5]}")  # Show first 5
            else:
                print(f"  {col}: No outliers detected")

print("\n✓ Data cleaning and validation complete!")

## Step 7: Final Data Summary

Generate a comprehensive summary of the cleaned dataset before saving.

In [ ]:
print("=" * 80)
print("FINAL CLEANED DATASET SUMMARY")
print("=" * 80)

print(f"\nDataset Shape: {df_clean.shape[0]} rows × {df_clean.shape[1]} columns")
print(f"Year Range: {df_clean['Year'].min()} to {df_clean['Year'].max()} ({df_clean['Year'].max() - df_clean['Year'].min() + 1} years)")

print("\nColumn Information:")
print("-" * 80)
for col in df_clean.columns:
    if col != 'Year':
        non_null = df_clean[col].dropna()
        print(f"\n{col.upper()}:")
        print(f"  - Non-null values: {len(non_null)} / {len(df_clean)} ({100*len(non_null)/len(df_clean):.1f}%)")
        if len(non_null) > 0:
            print(f"  - Range: [{non_null.min():.2f}, {non_null.max():.2f}]")
            print(f"  - Mean: {non_null.mean():.2f}")
            print(f"  - Median: {non_null.median():.2f}")

print("\n" + "=" * 80)
print("CLEANED DATA PREVIEW:")
print("=" * 80)
display(df_clean.head(10))
display(df_clean.tail(10))

print("\n" + "=" * 80)
print("DESCRIPTIVE STATISTICS:")
print("=" * 80)
display(df_clean.describe())

## Step 8: Save Cleaned Dataset

Save the cleaned dataset to the `datasets/cleaned/` directory for use in subsequent analysis stages.

In [ ]:
# Create output directory if it doesn't exist
output_dir = '../datasets/cleaned'
os.makedirs(output_dir, exist_ok=True)

# Define output file path
output_path = os.path.join(output_dir, 'ethiopia_analytic_dataset.csv')

# Save cleaned dataset
print(f"Saving cleaned dataset to: {output_path}")
df_clean.to_csv(output_path, index=False)

print("✓ Dataset saved successfully!")

# Verify the saved file
if os.path.exists(output_path):
    file_size = os.path.getsize(output_path) / 1024  # Size in KB
    print(f"  - File size: {file_size:.2f} KB")
    
    # Quick verification: reload and check
    df_verify = pd.read_csv(output_path)
    print(f"  - Verification: Loaded {df_verify.shape[0]} rows × {df_verify.shape[1]} columns")
    print(f"  - ✓ File saved and verified successfully!")
else:
    print("  - ✗ Error: File was not created!")

## Stage 1 Summary

### Completed Tasks
✅ **Data Acquisition**: Successfully loaded raw World Bank WDI data for Ethiopia  
✅ **Data Transformation**: Converted from wide format (years as columns) to long format, then pivoted to indicator columns  
✅ **Data Cleaning**: 
   - Converted data types appropriately
   - Handled missing values (preserved for downstream analysis)
   - Validated data ranges
   - Checked for outliers

✅ **Data Export**: Saved cleaned dataset to `datasets/cleaned/ethiopia_analytic_dataset.csv`

### Dataset Characteristics
- **Time Period**: 1960-2024 (65 years)
- **Indicators**: 5 key macroeconomic indicators
- **Format**: Year-indexed time series data
- **Quality**: Data validated and ready for analysis

### Next Steps
The cleaned dataset is now ready for:
- **Stage 2**: Exploratory Data Analysis (EDA)
- **Stage 3**: Linear Transformations
- **Stage 4**: Advanced Visualizations
- **Stage 5**: Feature Engineering
- **Stage 6**: Forecasting (ARIMA, Holt-Winters, ML models)
- **Stage 7**: Trend Analysis, Correlation Analysis, and Policy Insights

---
**Note**: Missing values are preserved in the cleaned dataset. They will be handled appropriately in downstream analysis stages based on the specific requirements of each modeling approach.